In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_qdrant import QdrantVectorStore
from langchain_ollama import OllamaEmbeddings
from langchain_qdrant import FastEmbedSparse, RetrievalMode
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank

import config

# Setup

In [2]:
url = config.QDRANT_URL
api_key = config.QDRANT_API_KEY


embeddings = OllamaEmbeddings(
    model="nomic-embed-text:v1.5"
)

sparse_embeddings = FastEmbedSparse(
    model_name="Qdrant/bm25"
)

qdrant = QdrantVectorStore.from_existing_collection(
    embedding=embeddings,
    sparse_embedding=sparse_embeddings,
    url=url,
    prefer_grpc=True,
    api_key=api_key,
    collection_name="mlops_document",
    retrieval_mode=RetrievalMode.HYBRID,
)

hybrid_rerank_qdrant_retriever = qdrant.as_retriever(
    search_type="similarity", 
    search_kwargs={"k": 20},
)

groq_llama3_1_70b = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    api_key=config.GROQ_API_KEY
)

Fetching 29 files:   0%|          | 0/29 [00:00<?, ?it/s]

# 1. Put Everything into one function

In [5]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def hybrid_search_reranker(
    query: str
) -> str:
    # -----------------------------------------------------------------
    # 1. Retrieval
    # -----------------------------------------------------------------

    compressor = FlashrankRerank(
        model="ms-marco-MiniLM-L-12-v2",
        top_n=4
    )
    
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor, base_retriever=hybrid_rerank_qdrant_retriever
    )

    retrieved_docs = compression_retriever.invoke(query)

    doc_context = format_docs(retrieved_docs)

    # -----------------------------------------------------------------
    # 2. Generation
    # -----------------------------------------------------------------
    # Prompt
    template = """
    You are an assistant for question-answering tasks. 
    Please answer the question based on the context provided.
    Do not tell the user that you are referring to the context to answer the question 
    If you don't know the answer or the context does not answer the question, just say that you don't know. 
    Elaborate your answer in well structured format.

    Question: {question} 
    Context: {context} 
    Answer:
    """

    prompt = ChatPromptTemplate.from_template(template)


    llm = groq_llama3_1_70b

    rag_chain = (
        prompt
        | llm
        | StrOutputParser()
    )

    return rag_chain.invoke({"question": query, "context":doc_context}), retrieved_docs, doc_context

# 2. Test and evaluate

## 2.1 Simple Query

In [8]:
question = "What is data distribution shifts ?"
result, retrieved_docs, doc_context = hybrid_search_reranker(question)
print(result)

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


**Data Distribution Shifts: Definition and Explanation**

Data distribution shifts refer to changes in the underlying distribution of the data, which can affect the performance of machine learning models. These shifts can occur in various forms, including concept drift, covariate shift, and label shift.

**Types of Data Distribution Shifts**

1. **Covariate Shift**: This occurs when the distribution of the input data (X) changes, but the conditional probability of the output given the input (P(Y|X)) remains the same. In other words, the input data distribution changes, but the relationship between the input and output remains constant.
2. **Label Shift**: This occurs when the distribution of the output data (Y) changes, but the conditional probability of the input given the output (P(X|Y)) remains the same.
3. **Concept Drift**: This occurs when the conditional probability of the output given the input (P(Y|X)) changes, but the input data distribution (P(X)) remains the same.

**Causes

## 2.2 Generate answer and retrieve the referred context

In [5]:
import pandas as pd
import joblib

hybrid_search_reranker_test = joblib.load(filename="test_cases.pkl")

INFO:numexpr.utils:NumExpr defaulting to 12 threads.


In [6]:
from tqdm import tqdm
import time

def answer_question_llm(
    test_cases: dict,
    question_key: str,
    answer_key: str,
    doc_context_key: str,
    rag_function
) -> dict:
    test_cases[answer_key] = []
    test_cases[doc_context_key] = []

    for question in tqdm(test_cases[question_key], desc="Answering questions", total=len(test_cases[question_key])):
        result, retrieved_docs, doc_context = rag_function(question)
        time.sleep(5)
        test_cases[answer_key].append(result)
        test_cases[doc_context_key].append(doc_context)
    return test_cases


In [7]:
hybrid_search_reranker_test = answer_question_llm(
    test_cases=hybrid_search_reranker_test,
    question_key="question",
    answer_key="hybrid_search_reranker_answer",
    doc_context_key="hybrid_search_reranker_doc_context",
    rag_function=hybrid_search_reranker
)

Answering questions:   0%|          | 0/20 [00:00<?, ?it/s]INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
Answering questions:   5%|▌         | 1/20 [00:10<03:25, 10.80s/it]INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
Answering questions:  10%|█         | 2/20 [00:21<03:13, 10.76s/it]INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
Answering questions:  15%|█▌        | 3/20 [00:32<03:06, 10.96s/it]INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:groq._base_c

In [8]:
print(hybrid_search_reranker_test['hybrid_search_reranker_answer'][0])

**Data Distribution Shifts**

Data distribution shifts refer to changes in the underlying distribution of the data, which can affect the performance of machine learning models. There are three distinct subtypes of data shifts: concept drift, covariate shift, and label shift.

**Types of Data Distribution Shifts**

1. **Covariate Shift**: This occurs when the distribution of the input data (P(X)) changes, but the conditional probability of an output given an input (P(Y|X)) remains the same. In other words, the input data changes, but the relationship between the input and output remains the same.

Example: In a breast cancer detection model, the input data may change if the model is trained on data from women over 40 and then deployed on data from women of all ages. However, the probability of having breast cancer given a certain age remains the same.

2. **Label Shift**: This occurs when the distribution of the output data (P(Y)) changes, but the conditional probability of an input giv

## 2.3 Deepeval RAG Evaluation

In [10]:
from langchain_groq import ChatGroq
from deepeval.models.base_model import DeepEvalBaseLLM
from deepeval.metrics import ContextualPrecisionMetric, ContextualRecallMetric, ContextualRelevancyMetric, AnswerRelevancyMetric, FaithfulnessMetric
from deepeval.test_case import LLMTestCase
from tqdm import trange

class GroqLLM(DeepEvalBaseLLM):
    def __init__(
        self,
        model
    ):
        self.model = model

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        return chat_model.invoke(prompt).content

    async def a_generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        res = await chat_model.ainvoke(prompt)
        return res.content

    def get_model_name(self):
        return "Custom Azure OpenAI Model"

# Replace these with real values
custom_model = ChatGroq(
    model="gemma2-9b-it",
    temperature=0,
    api_key=config.GROQ_API_KEY
)
# Define Critic model
groq_critic_llm = GroqLLM(model=custom_model)

# Define all metrics
contextual_precision = ContextualPrecisionMetric(
    threshold=0.7,
    model=groq_critic_llm,
    include_reason=False
)

contextual_recall = ContextualRecallMetric(
    threshold=0.7,
    model=groq_critic_llm,
    include_reason=False    
)
contextual_relevancy = ContextualRelevancyMetric(
    threshold=0.7,
    model=groq_critic_llm,
    include_reason=False
)

answer_relevancy = AnswerRelevancyMetric(
    threshold=0.7,
    model=groq_critic_llm,
    include_reason=False
)

faithfulness = FaithfulnessMetric(
    threshold=0.7,
    model=groq_critic_llm,
    include_reason=False
)


def evaluate_rag(
    rag_test_cases: dict,
    answer_relevancy_key: str,
    faithfulness_key: str,
    contextual_precision_key: str,
    contextual_recall_key: str,
    contextual_relevancy_key: str,
    question_key: str,
    expected_answer_key: str,
    actual_answer_key: str,
    doc_context_key: str
) -> dict:

    rag_test_cases[answer_relevancy_key] = []
    rag_test_cases[faithfulness_key] = []
    rag_test_cases[contextual_precision_key] = []
    rag_test_cases[contextual_recall_key] = []
    rag_test_cases[contextual_relevancy_key] = []

    # Evaluate each test case
    for i in tqdm(range(len(rag_test_cases[question_key])), desc="Evaluating answers", total=len(rag_test_cases[question_key])):
        question = rag_test_cases[question_key][i]
        expected_answer = rag_test_cases[expected_answer_key][i]
        actual_answer = rag_test_cases[actual_answer_key][i]
        doc_context = rag_test_cases[doc_context_key][i]


        test_case = LLMTestCase(
            input=question,
            actual_output=actual_answer,
            expected_output=expected_answer,
            retrieval_context=[
                doc_context
            ]
        )

        try:
            answer_relevancy.measure(test_case)
            answer_relevancy_score = answer_relevancy.score
        except:
            answer_relevancy_score = -1
        time.sleep(20)

        try:
            faithfulness.measure(test_case)
            faithfulness_score = faithfulness.score
        except:
            faithfulness_score = -1
        time.sleep(20)   

        try:
            contextual_precision.measure(test_case)
            contextual_precision_score = contextual_precision.score
        except:
            contextual_precision_score = -1
        time.sleep(20)

        try:
            contextual_recall.measure(test_case)
            contextual_recall_score = contextual_recall.score
        except:
            contextual_recall_score = -1
        time.sleep(20)

        try:
            contextual_relevancy.measure(test_case)
            contextual_relevancy_score = contextual_relevancy.score
        except:
            contextual_relevancy_score = -1
        time.sleep(20)

        rag_test_cases[answer_relevancy_key].append(answer_relevancy_score)
        rag_test_cases[faithfulness_key].append(faithfulness_score)
        rag_test_cases[contextual_precision_key].append(contextual_precision_score)
        rag_test_cases[contextual_recall_key].append(contextual_recall_score)
        rag_test_cases[contextual_relevancy_key].append(contextual_relevancy_score)

        time.sleep(5)

    return rag_test_cases





In [11]:
hybrid_search_reranker_test = evaluate_rag(
    rag_test_cases=hybrid_search_reranker_test,
    answer_relevancy_key="answer_relevancy",
    faithfulness_key="faithfulness",
    contextual_precision_key="contextual_precision",
    contextual_recall_key="contextual_recall",
    contextual_relevancy_key="contextual_relevancy",
    question_key="question",
    expected_answer_key="expected_answer",
    actual_answer_key="hybrid_search_reranker_answer",
    doc_context_key="hybrid_search_reranker_doc_context"
)

Evaluating answers:   0%|          | 0/20 [00:00<?, ?it/s]

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


...

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Evaluating answers: 100%|██████████| 20/20 [37:37<00:00, 112.86s/it]


In [27]:
hybrid_search_reranker_test.keys()

dict_keys(['question', 'expected_answer', 'query_type', 'hybrid_search_reranker_answer', 'hybrid_search_reranker_doc_context', 'answer_relevancy', 'faithfulness', 'contextual_precision', 'contextual_recall', 'contextual_relevancy'])

In [12]:
print(hybrid_search_reranker_test['hybrid_search_reranker_answer'][0])

**Data Distribution Shifts**

Data distribution shifts refer to changes in the underlying distribution of the data, which can affect the performance of machine learning models. There are three distinct subtypes of data shifts: concept drift, covariate shift, and label shift.

**Types of Data Distribution Shifts**

1. **Covariate Shift**: This occurs when the distribution of the input data (P(X)) changes, but the conditional probability of an output given an input (P(Y|X)) remains the same. In other words, the input data changes, but the relationship between the input and output remains the same.

Example: In a breast cancer detection model, the input data may change if the model is trained on data from women over 40 and then deployed on data from women of all ages. However, the probability of having breast cancer given a certain age remains the same.

2. **Label Shift**: This occurs when the distribution of the output data (P(Y)) changes, but the conditional probability of an input giv

In [13]:
import numpy as np

print(f"answer_relevancy: {np.average(hybrid_search_reranker_test['answer_relevancy'])}")
print(f"faithfulness: {np.average(hybrid_search_reranker_test['faithfulness'])}")
print(f"contextual_precision: {np.average(hybrid_search_reranker_test['contextual_precision'])}")
print(f"contextual_recall: {np.average(hybrid_search_reranker_test['contextual_recall'])}")
print(f"contextual_relevancy: {np.average(hybrid_search_reranker_test['contextual_relevancy'])}")

answer_relevancy: 0.8944444444444445
faithfulness: 0.986111111111111
contextual_precision: 0.4
contextual_recall: 0.9928571428571429
contextual_relevancy: 0.8066366256647587


In [22]:
import joblib
joblib.dump(value=hybrid_search_reranker_test, filename="hybrid_search_reranker_test.pkl")

['hybrid_search_reranker_test.pkl']